In [1]:
from DelphesUltility import *
from pylab import *
import os
from matplotlib.lines import Line2D

In [2]:
channel = 'eeg'

In [3]:
models = ["Gp3TeV_Th1TeV_Ph300GeV","Gp3TeV_Th1TeV_Ph400GeV","Gp3TeV_Th1TeV_Ph500GeV"]
        
# read dfs for models
pickleDir = BASEDIR+'/data/delphes/pickle'
dfsGen = [pd.read_pickle("{}/{}_{}_gen.pkl".format(pickleDir,model,channel)) for model in models]
dfs = [pd.read_pickle("{}/{}_{}.pkl".format(pickleDir,model,channel)) for model in models]

# load plotting config
configs = pd.read_csv(BASEDIR+'/data/delphes/csv/plotConfig.csv')

# create folders for saving plots
pathCombine = BASEDIR+"/plots/delphes_{}_combine".format(channel)
paths = [BASEDIR+"/plots/delphes_{}_{}".format(channel,model) for model in models]
for p in [pathCombine]+paths:
    if not os.path.exists(p): os.mkdir(p)

## 1. Plot for each model

In [4]:
def plot_split(config):
    v, a, b, step, xl = config
    mybin = np.arange(a,b,step)
    
    for i in range(len(models)):
        plt.figure(facecolor="w",figsize=(5,6))
        h = plt.hist(dfsGen[i][v], mybin, histtype='step', color="C0", lw=1, linestyle="-",
                 alpha=1.0,label="Gen n={}".format(len(dfsGen[i])))
        ynorm = h[0].max()
        plt.hist(dfs[i][v], mybin, histtype='stepfilled', color="C0", lw=2, linestyle="-",
                 alpha=0.6, label="Delphes w/ cuts n={}".format(len(dfs[i])))
        
        plt.xlabel(xl, fontsize=11)
        plt.ylim(0,1.3*ynorm)
        plt.grid(color='grey',linestyle="--",alpha=0.3)
        plt.legend(loc="best", fontsize=10)
        plt.title(models[i])
        plt.savefig(paths[i]+"/{}.png".format(v),dpi=300)
        plt.close()

In [5]:
for i, row in configs.iterrows():
    config = (row['v'], row['a'], row['b'], row['step'], row['xl'])
    plot_split(config)

## 2. Plot overlayed models

In [6]:
def plot_combine(config):
    v, a, b, step, xl = config
    mybin = np.arange(a,b,step)
    
    plt.figure(facecolor="w",figsize=(5,6))
    ax = plt.gca()
    
    ynorm = 0
    for i,model in enumerate(models):
        h = plt.hist(dfsGen[i][v], mybin, histtype='step', color="C"+str(i), lw=1, linestyle="--",
                 alpha=1.0,label="Gen n={}".format(len(dfsGen[i])))
        ynorm = max(ynorm, h[0].max())
        
        plt.hist(dfs[i][v], mybin, histtype='step', color="C"+str(i), lw=3, linestyle="-",
                 alpha=0.6, label="Delphes w/ cuts n={}".format(len(dfs[i])))
        
        plt.text(0.05,(19-i)/20, model, color="C"+str(i),transform=ax.transAxes)
    
    
    legend_elements = [Line2D([0], [0], color='k', lw=1, linestyle="--", label='Gen'),
        Line2D([0], [0], color='k', lw=3, linestyle="-", label='Delphes w/ cuts')]
    ax.legend(handles=legend_elements, loc='best')
    plt.xlabel(xl)
    plt.ylim(0,1.3*ynorm)
    plt.grid(color='grey',linestyle="--",alpha=0.3)
    plt.savefig(pathCombine+"/{}.png".format(v),dpi=300)
    plt.close()

In [7]:
for i, row in configs.iterrows():
    config = (row['v'], row['a'], row['b'], row['step'], row['xl'])
    plot_combine(config)